In [18]:
import gmaps
import time
import requests
import pandas as pd
import numpy as np
from citipy import citipy
import matplotlib.pyplot as plot
from config import weather_api_key
from config import g_key
from datetime import datetime
from scipy.stats import linregress
import matplotlib.pyplot as plt

In [19]:
city_data_df=pd.read_csv("../weather_data/cities.csv")
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [20]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [21]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 90
What is the maximum temperature you would like for your trip? 99


In [25]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
12,12,Santa Isabel Do Rio Negro,BR,2020-10-21 19:32:02,-0.41,-65.02,91.85,50,88,3.20
63,63,Xambioa,BR,2020-10-21 19:32:28,-6.41,-48.54,97.27,31,93,4.16
72,72,Santa Cruz,BO,2020-10-21 19:31:52,-17.80,-63.17,90.37,40,90,3.04
100,100,Atar,MR,2020-10-21 19:32:46,20.52,-13.05,95.00,28,56,6.93
103,103,Labrea,BR,2020-10-21 19:29:59,-7.26,-64.80,92.25,47,100,1.14
112,112,San Cristobal,VE,2020-10-21 19:32:51,7.77,-72.22,96.80,32,40,13.87
170,170,Jaru,BR,2020-10-21 19:33:24,-10.44,-62.47,90.28,48,100,0.89
204,204,Barra Do Corda,BR,2020-10-21 19:33:41,-5.50,-45.25,99.00,22,80,2.53
209,209,Lethem,GY,2020-10-21 19:33:43,3.38,-59.80,94.98,39,92,5.50
216,216,Caririacu,BR,2020-10-21 19:33:46,-7.04,-39.28,92.17,23,72,5.46


In [26]:
preferred_cities_df.count()

City_ID       22
City          22
Country       22
Date          22
Lat           22
Lng           22
Max Temp      22
Humidity      22
Cloudiness    22
Wind Speed    22
dtype: int64

In [27]:
preferred_cities_df=preferred_cities_df.dropna()

In [28]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
12,Santa Isabel Do Rio Negro,BR,91.85,-0.41,-65.02,
63,Xambioa,BR,97.27,-6.41,-48.54,
72,Santa Cruz,BO,90.37,-17.80,-63.17,
100,Atar,MR,95.00,20.52,-13.05,
103,Labrea,BR,92.25,-7.26,-64.80,
112,San Cristobal,VE,96.80,7.77,-72.22,
170,Jaru,BR,90.28,-10.44,-62.47,
204,Barra Do Corda,BR,99.00,-5.50,-45.25,
209,Lethem,GY,94.98,3.38,-59.80,
216,Caririacu,BR,92.17,-7.04,-39.28,


In [29]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
12,Santa Isabel Do Rio Negro,BR,91.85,-0.41,-65.02,
63,Xambioa,BR,97.27,-6.41,-48.54,
72,Santa Cruz,BO,90.37,-17.80,-63.17,
100,Atar,MR,95.00,20.52,-13.05,
103,Labrea,BR,92.25,-7.26,-64.80,
112,San Cristobal,VE,96.80,7.77,-72.22,
170,Jaru,BR,90.28,-10.44,-62.47,
204,Barra Do Corda,BR,99.00,-5.50,-45.25,
209,Lethem,GY,94.98,3.38,-59.80,
216,Caririacu,BR,92.17,-7.04,-39.28,


In [30]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [31]:
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping")

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


In [32]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
12,Santa Isabel Do Rio Negro,BR,91.85,-0.41,-65.02,Hotel GSA
63,Xambioa,BR,97.27,-6.41,-48.54,Hotel Rio Araguaia
72,Santa Cruz,BO,90.37,-17.80,-63.17,Hotel Cortez
100,Atar,MR,95.00,20.52,-13.05,Oumou Elghoura
103,Labrea,BR,92.25,-7.26,-64.80,Hotel Novo Horizonte
112,San Cristobal,VE,96.80,7.77,-72.22,Pirineos
170,Jaru,BR,90.28,-10.44,-62.47,Hotel Paraná
204,Barra Do Corda,BR,99.00,-5.50,-45.25,Belchior Palace Hotel
209,Lethem,GY,94.98,3.38,-59.80,Savannah Inn
216,Caririacu,BR,92.17,-7.04,-39.28,Chacara Emilia Cavalcante


In [33]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [35]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [36]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [37]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [39]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

AttributeError: module 'gmaps' has no attribute 'map'